## Example - 0D hydrogen CRM using CRM modelbound data and term generator functionality

A feature that adds flexibility and convenience to ReMKiT1D is associating term generators with custom models in addition to specifying user-generated terms. This notebook shows how to use the CRM modelbound data class in ReMKiT1D together with the CRM density evolution term generator to build a hydrogen CRM with inbuilt data. 

Initial data are designed to reproduce Figure 8 in Colonna et al. (Spectrochimica Acta Part B 56 2001 587᎐598) or to show convergence to Saha-Boltzmann for opaque plasmas.

This notebook is the rewrite of the original notebook for the tests performed in Section 5.3. in the ReMKiT1D code paper.

In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.crm_support as crm

### Some useful constants

In [ ]:
heavySpeciesMass = 2.014 #in amus
hPlanck = 6.62607004e-34
elMass =  9.10938e-31
elCharge = 1.60218e-19

### Wrapper initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_CRM_example/")

### Normalization setup

In [ ]:
norms = rk.norms

tempNorm = norms["eVTemperature"] 
densNorm = norms["density"]
timeNorm = norms["time"]


### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D
# Need a non-trivial velocity grid for <sigma v> integrals
vGrid = np.logspace(-2,0,80) #In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)

lMax = 0 
rk.grid = rmk.Grid(xGrid,vGrid,lMax,interpretXGridAsWidths=True ,interpretVGridAsWidths=True)

### Species initialization 

NOTE: The CRM density evolution generator assumes that all species it evolves have a density as their first associated variable and that it is an implicit and fluid variable. We shall use the v2.0.0 feature to associate variables with species later. Also, the prebuilt Janev data requires that neutral IDs correspond to principle quantum numbers of excited states.

In [ ]:
electronSpecies = rmk.Species("e",0) 
ionSpecies = rmk.Species("D+",-1)

rk.species.add(electronSpecies,ionSpecies)

numNeutrals=25
neutralDensList = ["n"+str(i) for i in range(1,numNeutrals+1)] # List of neutral density names
neutralSpecies = []
for neutral in neutralDensList:
    neutralSpecies.append(rmk.Species("D"+neutral[1:],int(neutral[1:]),heavySpeciesMass))
rk.species.add(*tuple(neutralSpecies))

### Variable initialization

In [ ]:
T0 = 1.72 # Temperature in eV approx 20000K corresponding to case 1 in Colonna et al.
T_i = T0*np.ones(rk.grid.numX)/tempNorm

T = rmk.Variable("T",rk.grid,data=T_i,units='$10eV$',isDerived=True)

rk.variables.add(T)

# 10% ionization fixed initial densities with no excited states
ne = 0.9
n1 = 0.1

ntot = ne + n1 #total density in case we want to calculate Saha-Boltzmann distribution

reprColonna = False # Set to true to approximately reproduce 10^-8 line in figure 8 of Colonna et al. If false will run to equilibrium at lower density
fixedID0 =  None # No fixed initial ionization degree
if reprColonna:
    ntot = 733893.9 # Density corresponding to approximately 1atm of pressure at 1000K
    fixedID0 = 1e-3

stateTempInit = T0/2 # (Saha-)Boltzmann temperature corresponding to case 1 in Colonna et al.
neutDensSBInit = [dens/densNorm for dens in crm.hydrogenSahaBoltzmann(numNeutrals,stateTempInit,ntot*densNorm,fixedIonizationDegree=fixedID0)]

initialSahaBoltzmann = True # Set to true for initial (Saha-)Boltzmann condition
n_i = ne*np.ones(rk.grid.numX)
if initialSahaBoltzmann: 
    n_i = neutDensSBInit[0]*np.ones(rk.grid.numX)  

ne = rmk.Variable("ne",rk.grid,data=n_i,units='$10^{19} m^{-3}$')
electronSpecies.associateVar(ne) # This is how we associate variables with species
ni = rmk.Variable("ni",rk.grid,data=n_i,units='$10^{19} m^{-3}$')
ionSpecies.associateVar(ni)

rk.variables.add(ne,ni)

# We need a distribution function to calculate rates from cross-sections built into the code
f_i = np.zeros([rk.grid.numX,rk.grid.numH,rk.grid.numV])
for i in range(rk.grid.numX):
    f_i[i,rk.grid.getH(0)-1,:] = (T_i[i]*np.pi)**(-1.5) * n_i[i] * np.exp(-rk.grid.vGrid**2/T_i[i])
if reprColonna:
    f = rmk.Variable("f",rk.grid,isDerived=True,isDistribution=True,data=f_i)
    
else:
    f = rmk.Variable("f",rk.grid,isDerived=True,isDistribution=True,derivation=rk.textbook["maxwellianDistribution"],derivationArgs=[T.name,ne.name],data=f_i)
rk.variables.add(f)
neutralVars = []

if initialSahaBoltzmann:
    for i in range(1,numNeutrals+1):
        neutralVars.append(rmk.Variable(neutralDensList[i-1],rk.grid,data=neutDensSBInit[i]*np.ones(rk.grid.numX),units='$10^{19} m^{-3}$'))
else:
    neutralVars.append(rmk.Variable("n1",rk.grid,data=n1*np.ones(rk.grid.numX),units='$10^{19} m^{-3}$'))
    for i in range(2,numNeutrals+1):
        neutralVars.append(rmk.Variable(neutralDensList[i-1],rk.grid,data=neutDensSBInit[i]*np.ones(rk.grid.numX),units='$10^{19} m^{-3}$'))

for i,var in enumerate(neutralVars):
    neutralSpecies[i].associateVar(var)
rk.variables.add(*tuple(neutralVars))
# Calculate expected Saha-Boltzmann at T0
neutDensSB = [dens/densNorm for dens in crm.hydrogenSahaBoltzmann(numNeutrals,T0,ntot*densNorm)]

#### Generating modelbound data based on built-in cross-sections

In [ ]:
includeRadiativeProcesses = reprColonna # Should be included for Colonna Figure 8 reproduction and turned off for Saha-Boltzmann convergence
#Set the list of included Janev transitions
includedJanevTransitions = ["ex","deex","ion","recomb3b"] 
if includeRadiativeProcesses:
    includedJanevTransitions.append("recombRad") #Add radiative recombination if radiative processes are included
mbData = crm.CRMModelboundData(rk.grid)
crm.addJanevTransitionsToCRMData(mbData,numNeutrals,tempNorm,f,T,detailedBalanceCSPriority=1,processes=includedJanevTransitions)

#### Reading and adding NIST data for spontaneous transition probabilities

Hydrogen transition probabilities are included in the Aki.csv file, and have been taken from the NIST database.

In [ ]:

"""
Kramida, A., Ralchenko, Yu., Reader, J., and NIST ASD Team (2021). NIST Atomic Spectra Database (ver. 5.9), [Online]. Available: https://physics.nist.gov/asd [2022, May 13]. National Institute of Standards and Technology, Gaithersburg, MD. DOI: https://doi.org/10.18434/T4W30F
"""

spontTransDict = crm.readNISTAkiCSV("../data/Aki.csv")

Adding hydrogen spontaneous emission transitions can be done using the following function.

In [ ]:
if includeRadiativeProcesses:
    crm.addHSpontaneousEmissionToCRMData(mbData,spontTransDict,min(numNeutrals,20),min(numNeutrals,20),timeNorm,tempNorm) #NIST data only has a full transition list for n<=20

#### Creating the CRM model object

The CRM model is constructed by specifying the modelbound data (defined above) and adding a term generator that can interpret that data and create a collisional-radiative model from it. 

Term generators are sets of rules used to automate term construction. The CRM term generator uses the modelbound CRM data of its host model, identifies which transitions produce a change in the populations of evolved species, and generates corresponding source and sink terms for each species. 

In [ ]:
#CRM model
 
model = rmk.Model(name="CRM",latexName="CRM")

model.setModelboundData(mbData)

model.addTermGenerator(crm.CRMTermGenerator("crmTermGen",[electronSpecies,ionSpecies]+neutralSpecies))

rk.models.add(model)

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
initialTimestep = 1e5 # Large timestep to obtain steady state
if reprColonna:
    initialTimestep = 1e-4 # Small timestep to resolve evolution for Colonna Fig 8 reproduction

In [ ]:
integrator = rmk.BDEIntegrator("BDE",nonlinTol=1e-12,absTol=100.0,convergenceVars=[f,neutralVars[0]])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models)
rk.integrationScheme = rmk.IntegrationScheme(dt=initialTimestep,steps=integrationStep)
rk.integrationScheme.setFixedNumTimesteps(1500,100)


#### Generate LaTeX summary PDF

In [ ]:
rk.generatePDF("CRM 0D Example")

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


#### Loading data


In [ ]:
loadedData = rk.loadSimulation()

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

#### Compare final state densities with a Saha-Boltzmann equilibrium

If reprColonna is true will compare to the initial distribution, otherwise will compare to the expected Saha-Boltzmann distribution at the electron temperature.

In [ ]:
dataset = loadedData.dataset

In [ ]:
# Neutral state densities extracted from dataset
neutralDens1 = [dataset[neutralDensList[i]][-1,0] for i in range(numNeutrals)]

# Excited state energies
stateEnergies = [13.6*(1-1/(i+1)**2) for i in range(numNeutrals)]

if reprColonna:
    coords1 = [(stateEnergies[i],neutralDens1[i]/((i+1)**2)/(neutralDens1[0])) for i in range(numNeutrals)] #Degeneracy weighted densities in final timestep
    
    coords2 = [(stateEnergies[i],neutDensSBInit[i+1]/((i+1)**2*neutDensSBInit[1]) )for i in range(numNeutrals)] #Initial Saha-Boltzmann densities (weighted)
else:
    coords1 = [(stateEnergies[i],neutralDens1[i]/(2*(i+1)**2)) for i in range(numNeutrals)] #Degeneracy weighted densities in final timestep
    coords2 = [(stateEnergies[i],neutDensSB[i+1]/(2*(i+1)**2)) for i in range(numNeutrals)] #Expected Saha-Boltzmann densities (weighted)
    
if reprColonna: 
    label2 = 't = 0'
    label1 = f't = {dataset.coords["t"].values[-1]*timeNorm:.2e}s'
else:
    label1 = 'ReMKiT1D'
    label2 = 'Saha-Boltzmann'
    
curve2 = hv.Curve(coords2,label=label2).opts(color="r")
if reprColonna:
    curve1 = hv.Curve(coords1,label=label1).opts(color="k",linestyle='--')
else:
    curve1 = hv.Scatter(coords1,label=label1).opts(marker="x",color="k",s=15.0)
    
curve = curve2*curve1
curve.opts(logy=True,xlabel="E [eV]",ylabel='$n_i/g_i$',aspect=0.5)


#### Produce paper plots

In [ ]:
if reprColonna:
    hv.save(curve.opts(logy=True,xlabel="E [eV]",ylabel='$n_i/g_i$',aspect=0.5),"colonna_comp.pdf",dpi=144)
else:
    hv.save(curve.opts(logy=True,xlabel="E [eV]",ylabel='$n_i/g_i$',aspect=0.5),"sb_comp.pdf",dpi=144)